# NLP and entity extraction
## Week 10
### Summary

Use the Stanford NLP tools to identify named entities in a short text, then write a Python program to parse the NLP output.

### Details

The text in question is one we've already seen (in the sutructured data exercise). To review: it's a brief pamphlet, *Growler's Income Tax* (1864), by the prolific mid-nineteenth-century writer T.S. Arthur. It's a defense of the then-new income tax, instituted in 1861 to fund the Union's war effort. As you'll see, the text is pretty straightforward, but it's kind of nifty (or infuriating, I guess) how similar are the arguments it presents concerning taxation to those you might hear today. Go ahead, download the [plain-text copy](https://github.com/wilkens/course-exercises-f15/blob/master/arthur.txt) and read it now, if you haven't already. It's short. (Note that you could also work with the [XML version](https://github.com/wilkens/course-exercises-f15/blob/master/growler.xml) from the previous exercise and convert that to plain text as shown in the [solution to that exercise](https://github.com/wilkens/course-exercises-f15/blob/master/Parsing%20structured%20data%20solution.ipynb).)

Anyway, tax policy isn't really the point. Your task is to identify algorithmically the named entities in the text and to extract them for further processing. To do this, you'll use the Stanford NLP toolset. One possibility would be to use the full CoreNLP package, with which you can do much more than identify named entities (for example, you can do part of speech tagging, co-reference resolution, and sentiment analysis) in a single pass over a given text. But the simplest approach is to use just the [NER tool](http://nlp.stanford.edu/software/CRF-NER.shtml).

The NER tool, like the full CoreNLP package, is a Java program. That means you’ll need to have Java on your computer. And not just the Java runtime for your browser; you need the full [Java Development Kit](http://www.oracle.com/technetwork/java/javase/downloads/jdk8-downloads-2133151.html). Make sure you get the proper version for your system (OS and 32/64-bit; you do not need the "Demos and Samples Downloads"). You can check to see if you have Java installed by opening a terminal and typing `java -version`. If you get something like these lines (the Stanford tools require version 1.8 or higher), you’re good:

```
java version "1.8.0_60"
Java(TM) SE Runtime Environment (build 1.8.0_60-b27)
Java HotSpot(TM) 64-Bit Server VM (build 25.60-b23, mixed mode)
```

Follow the instructions included with the NER download ( see `README.txt`) to process the input text. Note that there are several output formats supported. I'd suggest using `-outputFormat inlineXML`, which I believe is the default. Save your output to an appropriately named file.

Now write some Python code that reads the NER output file and builds a list of unique entities in the output, each entity's type, and a count of how many times each entity occurs.

Your program should print a summary of this data to standard output and write a CSV file named `entities.csv` that contains the same information. Your terminal output (with made-up data) should look roughly like this:

```
Entity		Type		Count
------		----		-----
Boston		Location	2
John Smith	Person		1
```

Your CSV file should have the same structure, but no fake underlining and no tabs, with entities separated by commas. For an optional challenge, make sure your output can accommodate entity strings that include commas (probably by making sure all entity strings are enclosed by quotation marks or other markers that indicate the beginning and end of strings).

### Alternative processing
If you're up for a modest challenge, you might try to invoke the the NER tool from within Python using the NLTK module (see the [`nltk.tag.stanford`](http://www.nltk.org/api/nltk.tag.html#module-nltk.tag.stanford) docs) rather than from the command line. Or, if you really want, you could skip the Stanford tools entirely and use NLTK's (slower, lower-performing, but pure Python) named entity chunker. But both ideas are strictly optional.

### Submit

Submit three files via Sakai: your Python code as an iPython notebook, your NER output (in whatever format, probably ill-formed XML), and your CSV entity output.

### Consider

A few things to think about before class:

* How well or poorly do the entities extracted from the text square with your sense of what the text is about, whom it involves, and where it occurs (or with what areas it’s concerned)?
* How accurate is the NER process?
* How might you try to improve NER accuracy?

## Starter code

Here's a bit of code to get you going ...

In [3]:
from bs4 import BeautifulSoup
import csv

input_name =  "arthur-ner.xml"      # File names for input and output
output_name = "entities.csv"

with open(input_name, 'r') as f:
    soup = BeautifulSoup(f, 'lxml')
    #print(soup.prettify()) # Can use this to examine the parsed file
    
    locations = {}  # Dictionaries to hold found entities and their counts
    people    = {}  # Could also use a dict of dicts.
    orgs      = {}
    
    for i in soup.p.find_all(): # NB. finding subtags of 'p', inserted by BeautifulSoup
        entity_name = i.get_text()
        entity_type = i.name
        
        ## Your code here ##
        pass

In [4]:
# The remaining code ...
def incrementEntity(entity_string, dictionary):
    """ Examines dict 'dictionary' for key 'entity_string'.
        If key exists, increments counter value, else creates key and
        assigns value 1."""
    try:
        dictionary[entity_string] += 1
    except KeyError:
        dictionary[entity_string] = 1

def outputResults(dictionary, entity_type, f):
    """ Takes input dict 'dictionary', str 'entity_type',
        and csv.writer pointer 'f'.
        Sorts dictionary keys by values highest to lowest, and writes
        formatted output to CSV and to screen."""
    for i in sorted(dictionary, key=dictionary.get, reverse=True):
        print(i, '\t', entity_type, '\t', dictionary[i])
        f.writerow([i, entity_type, dictionary[i]])

# Re-initialize variables
locations = {}  # Dictionaries to hold found entities and their counts
people    = {}  # Could also use a dict of dicts.
orgs      = {}

# Loop over all tags in input file. Assumes only NER tags are present.
for i in soup.p.find_all():
    entity_name = i.get_text()
    entity_type = i.name
    # Note code below assumes we know the full list of entity types
    #  More versatile would be to build a dict of dicts, i.e.:
    #   entities = {"London":{"location":250, "person":3}}
    #  and then iterate over that. Meh, overkill here.
    if entity_type == 'location':
        incrementEntity(entity_name, locations)
    elif entity_type == 'person':
        incrementEntity(entity_name, people)
    elif entity_type == 'organization':
        incrementEntity(entity_name, orgs)
    else:
        continue

with open(output_name, 'w') as output_file:
    f = csv.writer(output_file)
    print("Entity\t\tType\tCount")
    print("------\t\t----\t-----")
    f.writerow(["Entity", "Type", "Count"])

    outputResults(locations, 'location', f)
    outputResults(people, 'person', f)
    outputResults(orgs, 'organization', f)

Entity		Type	Count
------		----	-----
Pennsylvania 	 location 	 2
Gettysburg 	 location 	 1
Philadelphia 	 location 	 1
Rappahannock 	 location 	 1
Delaware 	 location 	 1
France 	 location 	 1
Florida 	 location 	 1
Carlisle 	 location 	 1
Alabama 	 location 	 1
District 	 location 	 1
England 	 location 	 1
Lee 	 person 	 2
JOHN M. RILEY 	 person 	 2
ARTHUR 	 person 	 1
Growler 	 person 	 1
Riley 	 person 	 1


### The NLTK version

In [5]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.chunk import tree2conlltags, conlltags2tree
from nltk import pos_tag, ne_chunk
from collections import defaultdict

counts = defaultdict(lambda: defaultdict(int))

with open('arthur.txt', 'r') as f:
    text = f.read()
    sentences = sent_tokenize(text)
    for sent in sentences:
        tokens = word_tokenize(sent)
        pos = pos_tag(tokens)
        ne = ne_chunk(pos)
        ne_tags = tree2conlltags(ne)
        for entity in ne_tags:
            if entity[2] != 'O':
                counts[entity[2]][entity[0]] += 1 

/Users/mwilkens/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/utils/fixes.py:64: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  if 'order' in inspect.getargspec(np.copy)[0]:


In [6]:
for i in sorted(counts.keys()):
    print('\n',i)
    for j in sorted(counts[i], key=counts[i].get, reverse=True):
        print('  ', j, counts[i][j])


 B-GPE
   Pennsylvania 2
   Southern 1
   ARTHUR 1
   France 1
   Carlisle 1
   GROWLER 1
   District 1
   American 1
   Income 1
   Gettysburg 1
   South 1
   Rappahannock 1
   Collector 1
   Florida 1
   Stand 1
   England 1

 B-GSP
   Philadelphia 1

 B-ORGANIZATION
   INCOME 1
   Assessor 1
   Delaware 1
   RICHARD 1
   Alabama 1
   Union 1
   Income 1

 B-PERSON
   Growler 13
   Lee 2
   JOHN 2
   Collector 2
   Tax 1

 I-ORGANIZATION
   Tax 1

 I-PERSON
   M. 2
   RILEY 2
   Riley 1
   Revenue 1
   Internal 1
